In [16]:
#!pip install autogluon
# !pip uninstall pandas
# !pip install pandas

In [17]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import numpy as np
import random
import numpy as np

In [18]:

# Set the random seed for Python, NumPy, MXNet, and PyTorch (if you're using them)
seed_value = 42  # Replace with your desired seed value
random.seed(seed_value)
np.random.seed(seed_value)

In [19]:
x_train_b = pd.read_csv('cleaned_data/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data/B/x_test_b.csv')

x_train_b

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,month_sin,month_cos,day_of_year,day_of_year_sin,day_of_year_cos,direct_rad_3h_roll_avg,diffuse_rad_3h_roll_avg,direct_rad_6h_roll_avg,diffuse_rad_6h_roll_avg,direct_rad_x_sun_elevation
0,2019-01-01 00:00:00,5.525,1.23975,1200.6750,0.0,0.000,1200.6750,0.0,275.150,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
1,2019-01-01 01:00:00,5.425,1.23975,1131.4249,0.0,0.000,1131.4249,0.0,274.825,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
2,2019-01-01 02:00:00,5.400,1.23850,1061.0000,0.0,0.000,1061.0000,0.0,274.800,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
3,2019-01-01 03:00:00,5.350,1.23975,1021.1500,0.0,0.000,1021.1500,0.0,274.675,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
4,2019-01-01 04:00:00,5.675,1.23750,1033.7000,0.0,0.000,1033.7000,0.0,275.500,0.000,...,0.500000,0.866025,1,0.017166,0.999853,0.000000,0.000000,0.000000,0.000000,-0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26956,2023-04-30 19:00:00,4.550,1.27650,1677.9500,337850.1,4.225,542.8500,0.0,272.425,2.825,...,0.866025,-0.500000,120,0.882679,-0.469977,20.058333,35.033333,83.608334,94.829166,-0.01805
26957,2023-04-30 20:00:00,4.500,1.27875,1766.5000,9083.1,0.000,546.3500,0.0,272.300,0.000,...,0.866025,-0.500000,120,0.882679,-0.469977,2.458333,10.458333,52.808334,63.945835,-0.00000
26958,2023-04-30 21:00:00,4.500,1.27900,1698.9250,0.0,0.000,548.0500,0.0,272.300,0.000,...,0.866025,-0.500000,120,0.882679,-0.469977,0.033333,0.941667,26.433334,37.866667,-0.00000
26959,2023-04-30 22:00:00,4.500,1.27975,1354.8250,0.0,0.000,527.6000,0.0,272.275,0.000,...,0.866025,-0.500000,120,0.882679,-0.469977,0.000000,0.000000,10.029167,17.516666,-0.00000


In [20]:
x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)

train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'day_of_year','date_forecast'], axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour','hour_sin','hour_cos','month_sin','month_cos','day_of_year_sin','day_of_year_cos','direct_rad_3h_roll_avg','diffuse_rad_3h_roll_avg','direct_rad_6h_roll_avg','diffuse_rad_6h_roll_avg', 'day_of_year','date_forecast'], axis = 1)

In [21]:
test_data_b

,location,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation,observed
0,B,4.300,1.28300,912.3000,0.0,0.000,1059.750,0.0,271.65002,0.000,...,3.950,2.100,3.375,0.0,2023,5,1,0,-0.000000,0
1,B,4.250,1.28300,1482.8002,0.0,0.000,1073.700,0.0,271.45000,0.000,...,3.825,1.925,3.300,0.0,2023,5,1,1,-0.000000,0
2,B,4.150,1.28275,1765.9000,0.0,0.000,1200.100,0.0,271.05000,0.000,...,3.650,1.750,3.225,0.0,2023,5,1,2,-0.000000,0
3,B,4.025,1.28225,2269.7500,40510.2,11.675,1179.000,0.0,270.65000,9.375,...,3.500,1.475,3.150,0.0,2023,5,1,3,2.967300,0
4,B,3.900,1.28200,2198.2250,567057.1,76.900,919.150,0.0,270.37500,47.400,...,3.325,1.300,3.075,0.0,2023,5,1,4,190.086060,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,B,8.350,1.19800,3640.1250,1908360.9,85.100,2015.750,0.0,281.57500,33.625,...,2.475,2.075,-1.350,0.0,2023,7,3,19,109.454025,0
716,B,8.525,1.20075,3351.1000,737351.8,24.800,1613.375,0.0,281.85000,14.350,...,2.450,2.100,-1.275,0.0,2023,7,3,20,11.602287,0
717,B,8.800,1.20375,2753.0250,149728.8,1.275,1624.450,0.0,282.30000,1.300,...,2.575,2.150,-1.400,0.0,2023,7,3,21,-0.000000,0
718,B,9.000,1.20600,2204.5000,1440.5,0.000,1768.325,0.0,282.67502,0.000,...,2.250,1.800,-1.350,0.0,2023,7,3,22,-0.000000,0


# Trying with validation set equal to half of the estimated data


In [22]:
def split_dataset(train_data, observed_column='observed'):
    """
    Splits the dataset into a training set and a validation set.
    The validation set contains the last half of the rows where observed = 0,
    and the training set contains the rest.

    :param train_data: The original training dataset as a pandas DataFrame.
    :param observed_column: The name of the column that indicates if the row is observed.
    :return: A tuple (training_set, validation_set)
    """

    # Filter rows where observed = 0
    observed_zero = train_data[train_data[observed_column] == 0]

    # Split the filtered dataset into two
    half_index = len(observed_zero) // 2
    validation_set = observed_zero[half_index:]

    # Combine the first half of observed_zero with the rest of the data where observed != 0
    training_set = pd.concat([train_data[train_data[observed_column] != 0], observed_zero[:half_index]])

    return training_set, validation_set


# Example usage:
# train_data_a, val_data_a = split_dataset(train_data_a, 'observed')
# train_data_b, val_data_b = split_dataset(train_data_b, 'observed')
# train_data_c, val_data_c = split_dataset(train_data_c, 'observed')


In [23]:

#val_data_a.head()

In [24]:
predictor_b= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_b, verbosity = 2,presets='best_quality', time_limit= 100)

No path specified. Models will be saved in: "AutogluonModels\ag-20231107_174254\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 100s
AutoGluon will save models to "AutogluonModels\ag-20231107_174254\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   686.10 GB / 1022.87 GB (67.1%)
Train Data Rows:    26961
Train Data Columns: 51
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, -0.0, 103.38575, 209.32128)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['bi

[1000]	valid_set's l1: 21.7464
[2000]	valid_set's l1: 20.8121
[3000]	valid_set's l1: 20.3777


	Ran out of time, early stopping on iteration 3472. Best iteration is:
	[3472]	valid_set's l1: 20.2582


[1000]	valid_set's l1: 23.1174
[2000]	valid_set's l1: 22.2149
[3000]	valid_set's l1: 21.7002


	Ran out of time, early stopping on iteration 3394. Best iteration is:
	[3392]	valid_set's l1: 21.5744


...

	Ran out of time, early stopping on iteration 3872. Best iteration is:
	[3871]	valid_set's l1: 20.8633


[1000]	valid_set's l1: 22.8082
[2000]	valid_set's l1: 21.8512
[3000]	valid_set's l1: 21.3391


	Ran out of time, early stopping on iteration 4067. Best iteration is:
	[4063]	valid_set's l1: 21.0532


[4000]	valid_set's l1: 21.0692
[1000]	valid_set's l1: 24.1759
[2000]	valid_set's l1: 23.1695
[3000]	valid_set's l1: 22.6913
[4000]	valid_set's l1: 22.3854


	Ran out of time, early stopping on iteration 4563. Best iteration is:
	[4563]	valid_set's l1: 22.2652
	-21.1731	 = Validation score   (-mean_absolute_error)
	60.43s	 = Training   runtime
	1.1s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 1.27s of the 34.52s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
		[Errno 203] AssignProcessToJobObject() failed
Detailed Traceback:
Traceback (most recent call last):
  File "C:\Users\holwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\autogluon\core\trainer\abstract_trainer.py", line 1733, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "C:\Users\holwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\autogluon\core\train

In [25]:
predictor_b.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-21.155342,3.203264,88.605553,0.001013,0.167526,3,True,7
1,WeightedEnsemble_L2,-21.173140,1.098195,60.630297,0.000000,0.195564,2,True,4
2,LightGBMXT_BAG_L1,-21.173140,1.098195,60.434734,1.098195,60.434734,1,True,3
3,RandomForestMSE_BAG_L2,-21.454187,3.159251,71.884020,0.585751,11.330090,2,True,5
4,CatBoost_BAG_L2,-21.477477,2.616500,77.107938,0.043000,16.554007,2,True,6
5,KNeighborsUnif_BAG_L1,-40.354606,0.743909,0.063197,0.743909,0.063197,1,True,1
6,KNeighborsDist_BAG_L1,-40.614342,0.731396,0.056000,0.731396,0.056000,1,True,2


In [26]:
nan_counts = x_train_b_combined.isna().sum()
print(nan_counts)

date_forecast                    0
absolute_humidity_2m:gm3        24
air_density_2m:kgm3             24
ceiling_height_agl:m          4229
clear_sky_energy_1h:J            0
                              ... 
diffuse_rad_6h_roll_avg         19
direct_rad_x_sun_elevation      24
time                             0
pv_measurement                   0
observed                         0
Length: 69, dtype: int64


In [27]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b#print the DataFrame

,pv_measurement
0,0.217333
1,0.293051
2,0.304791
3,5.840382
4,56.086708
...,...
715,43.715218
716,14.703658
717,2.917425
718,0.290265


In [28]:
y_pred_b.to_csv('autogluon_b')

In [29]:
import os
output_dir = 'autogluon'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the dataframes into the folder
y_pred_b.to_csv(os.path.join(output_dir, 'y_pred_b3.csv'))

In [30]:

residuals = y_train_a - pred_a_whole
residuals_with_time = residuals.rename(columns= {'pv_measurement':'residuals'})
residuals_with_time['time'] = time_values
residuals_with_time

residuals_with_time.set_index('time')

NameError: name 'y_train_a' is not defined

,residuals
time,
2019-06-03 08:00:00,847.536738
2019-06-03 09:00:00,719.366240
2019-06-03 10:00:00,-586.029551
2019-06-03 11:00:00,-1148.709365
2019-06-03 18:00:00,-642.441157
...,...
2023-04-30 10:00:00,-1210.288330
2023-04-30 12:00:00,1156.401699
2023-04-30 13:00:00,1197.584092


,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,wind_speed_w_1000hPa:ms,forecast_year,forecast_month,forecast_day,forecast_hour,direct_rad_x_sun_elevation,pv_measurement,observed,time,residuals
0,9.850000,1.20250,654.42500,8968757.0,674.70000,654.42500,0.0,284.15,215.62500,3119993.8,...,0.0,2019,6,3,8,518.5806,2189.88,1,2019-06-03 08:00:00,847.536738
1,10.075001,1.19850,997.60004,10340040.0,756.55000,997.60004,0.0,284.50,265.57500,3464514.2,...,0.0,2019,6,3,9,2670.3330,3047.22,1,2019-06-03 09:00:00,719.366240
2,9.975000,1.18950,1940.00000,11294876.0,806.47500,1940.00000,0.0,284.50,267.75000,3839773.8,...,0.0,2019,6,3,10,13069.3125,2163.26,1,2019-06-03 10:00:00,-586.029551
3,9.900000,1.18000,2910.65000,11761468.0,820.94995,2910.65000,0.0,284.50,199.82500,3366538.0,...,0.0,2019,6,3,11,21901.4220,2686.64,1,2019-06-03 11:00:00,-1148.709365
4,10.725000,1.19925,1605.57500,3192082.2,162.32500,1605.57500,0.0,285.45,56.37500,967743.0,...,0.0,2019,6,3,18,738.1957,340.12,1,2019-06-03 18:00:00,-642.441157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5447,5.275000,1.26525,1910.70000,9674970.0,695.00000,496.09998,0.0,274.50,294.10000,4123328.5,...,0.0,2023,4,30,10,5100.8240,346.50,0,2023-04-30 10:00:00,-1210.288330
5448,5.250000,1.26200,2773.72500,10096505.0,686.10000,424.40000,0.0,274.60,256.72500,3519637.0,...,0.0,2023,4,30,12,2697.4460,4318.16,0,2023-04-30 12:00:00,1156.401699
5449,4.950000,1.26175,3951.92500,9486127.0,625.60000,627.32500,0.0,273.70,253.97499,3677273.2,...,0.0,2023,4,30,13,3971.2922,3374.14,0,2023-04-30 13:00:00,1197.584092
5450,4.725000,1.26275,4965.75000,8377467.0,533.12500,736.87500,0.0,273.05,185.32500,3162812.0,...,0.0,2023,4,30,14,5807.9400,2170.74,0,2023-04-30 14:00:00,820.224253


SyntaxError: EOL while scanning string literal (1256647608.py, line 23)